In [1]:
import sys
import asyncio
import logging

import numpy as np

from lsst.ts import salobj

import matplotlib.pyplot as plt

from lsst.ts.externalscripts.auxtel.latiss_cwfs_align import LatissCWFSAlign

In [2]:
stream_handler = logging.StreamHandler(sys.stdout)

logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [3]:
cwfs = LatissCWFSAlign(index=1)

In [4]:
await cwfs.start_task

Read historical data in 0.01 sec
Read historical data in 0.27 sec
Read historical data in 1.10 sec
Read historical data in 1.25 sec
Read historical data in 1.34 sec
Read historical data in 1.34 sec
Read historical data in 1.88 sec
Read historical data in 2.40 sec
Read historical data in 3.19 sec
Read historical data in 4.33 sec
Read historical data in 5.73 sec
Read historical data in 0.03 sec
RemoteTelemetry(ATDome, 0, position) falling behind; read 17 messages


In [20]:
await salobj.set_summary_state(cwfs.latiss.atspectrograph, salobj.State.ENABLED, settingsToApply="current")

[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]

In [8]:
await cwfs.attcs.stop_tracking()

Stop tracking.
Mount tracking state is 9
Tracking state: 10.
Tracking state: 8.
In Position: True.
In Position: False.


In [9]:
await cwfs.attcs.atptg.cmd_focusName.set_start(focus=3)

In [10]:
await cwfs.attcs.slew_object(name="HD 56618", rot_sky=0.)

Slewing to HD 56618: 07 16 34.9931 -27 52 52.245
Sending command
Stop tracking.
Mount tracking state is 8
In Position: False.
Scheduling check coroutines
process as completed...
Got False
Telescope not in position
atmcs: <State.ENABLED: 2>
atptg: <State.ENABLED: 2>
ataos: <State.ENABLED: 2>
atpneumatics: <State.ENABLED: 2>
athexapod: <State.ENABLED: 2>
atdome: <State.ENABLED: 2>
atdometrajectory: <State.ENABLED: 2>
[Telescope] delta Alt = +000.305 | delta Az= -000.160 [Dome] delta Az = -002.950
[Telescope] delta Alt = +000.132 | delta Az= -000.113 [Dome] delta Az = -002.950
[Telescope] delta Alt = +000.132 | delta Az= -000.073 [Dome] delta Az = -002.950
[Telescope] delta Alt = +000.131 | delta Az= -000.072 [Dome] delta Az = -002.950
[Telescope] delta Alt = +000.131 | delta Az= -000.072 [Dome] delta Az = -002.950
[Telescope] delta Alt = +000.131 | delta Az= -000.072 [Dome] delta Az = -002.950
[Telescope] delta Alt = +000.131 | delta Az= -000.072 [Dome] delta Az = -002.950
Got True
Waiti

In [21]:
await cwfs.latiss.take_engtest(exptime=5., n=1, filter='KPNO_406_828nm', grating='empty_1')

Generating group_id
ENGTEST 0001 - 0001
Waiting for header service LFO before continuing


array([2020021800027])

In [15]:
await cwfs.latiss.atarchiver.evt_heartbeat.next(flush=True)

In [ ]:
await cwfs.latiss.take_object(exptime=5., n=1)

In [ ]:
await asyncio.gather(cwfs.latiss.take_engtest(exptime=30., n=1),
                     cwfs.attcs.slew_object(name="HD 83832", rot_sky=-90.),
                     return_exceptions=True)

In [ ]:
await asyncio.gather(cwfs.latiss.take_engtest(exptime=3., n=5),
                     cwfs.attcs.slew_object(name="HD 83832", rot_sky=0.),
                     return_exceptions=True)

In [ ]:
await cwfs.latiss.take_object(exptime=5., n=1)

In [ ]:
await cwfs.attcs.offset_xy(x=-30., y=0.)

In [ ]:
await cwfs.latiss.take_object(exptime=5.,n=1)

In [23]:
cwfs.exposure_time

30.0

In [ ]:
await cwfs.hexapod_offset(offset=1.5)

In [5]:
cwfs.intra_visit_id = 2020021700284
cwfs.extra_visit_id = 2020021700285

In [12]:
cwfs.angle = -cwfs.camera_rotation_angle

In [22]:
await cwfs.run_cwfs()

Intra/Extra images not taken. Running take image sequence.
ENGTEST 0001 - 0001
Waiting for header service LFO before continuing
ENGTEST 0001 - 0001
Waiting for header service LFO before continuing
intraImage expId for target: 2020021800028
extraImage expId for target: 2020021800029


/home/saluser/repos/ts_externalscripts/python/lsst/ts/externalscripts/auxtel/latiss_cwfs_align.py:306: FutureWarning: Config field doAddDistortionModel is deprecated: Camera geometry is incorporated when reading the raw files. This option no longer is used, and will be removed after v19.
  isrConfig.doAddDistortionModel = False


Could not get intra focus image from butler. Waiting 5.0s and trying again.
RemoteEvent(ATMCS, 0, target) falling behind; read 11 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 12 messages


UnboundLocalError: local variable 'data_ref' referenced before assignment

In [13]:
cwfs.show_results()

Measured zernike coeficients: [-35.67100101635535, 11.732558498870029, -446.8696685742186]
De-rotated zernike coeficients: [ -35.67100102   11.7325585  -446.86966857]
Hexapod offset: [0.27229772 0.08956152 0.10639754]
Telescope offsets: [16.33786306  5.37369092  0.        ]



In [10]:
fig1 = plt.figure(1, figsize=(12,8))

ax11 = fig1.add_subplot(121)

ax11.set_title("defocus 0.8 - intra")
ax11.imshow(cwfs.I1[0].image0)
ax11.contour(cwfs.algo.pMask) 

ax12 = fig1.add_subplot(122)

ax12.set_title("defocus 0.8 - extra")
ax12.imshow(cwfs.I2[0].image0)
ax12.contour(cwfs.algo.pMask) 

In [ ]:
fig1 = plt.figure(1, figsize=(12,8))

ax = fig1.add_subplot(111)

ax.imshow(cwfs.detection_exp.image.array)

In [21]:
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend("matplotlib")

%matplotlib widget

plt.rcParams['figure.figsize'] = [7, 6]

disp = afwDisplay.Display(1, reopenPlot=True)

disp.scale("asinh", "zscale")
disp.mtv(cwfs.detection_exp)

Loaded backend module://ipympl.backend_nbagg version unknown.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

locator: <matplotlib.colorbar._ColorbarAutoLocator object at 0x7efb881a4240>
Using fixed locator on colorbar
Setting pcolormesh


In [14]:
await cwfs.latiss.atheaderservice.cmd_disable.start()

In [15]:
await cwfs.latiss.atheaderservice.cmd_standby.start()

In [16]:
await cwfs.latiss.atheaderservice.cmd_exitControl.start()

In [ ]:
await cwfs.attcs.atmcs.cmd_startTracking.start()

In [17]:
await cwfs.latiss.enable()

Gathering settings.
No settings for atcamera.
Using  from settingVersions event.
No settings for atspectrograph.
Using current from settingVersions event.
No settings for atheaderservice.
Couldn't get settingVersions event. Using empty settings.
No settings for atarchiver.
Using normal from settingVersions event.
Settings versions: {'atcamera': '', 'atspectrograph': 'current', 'atheaderservice': '', 'atarchiver': 'normal'}
Enabling all components
Enabling  atcamera
Enabling  atspectrograph
Enabling  atheaderservice
Enabling  atarchiver
[atcamera]::[<State.ENABLED: 2>]
[atspectrograph]::[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]
[atheaderservice]::[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]
[atarchiver]::[<State.ENABLED: 2>]
All components enabled.


In [18]:
await cwfs.latiss.take_darks?

Signature: cwfs.latiss.take_darks(exptime, ndarks, group_id=None, checkpoint=None)
Docstring:
Take a series of dark images.

Parameters
----------
exptime : `float`
    Exposure time for darks.
ndarks : `int`
    Number of dark frames to take.
group_id : `str`
    Optional group id for the data sequence. Will generate a common
    one for all the data if none is given.
checkpoint : `coro`
    A optional awaitable callback that accepts one string argument
    that is called before each bias is taken.
File:      ~/repos/ts_standardscripts/python/lsst/ts/standardscripts/auxtel/latiss.py
Type:      method


In [19]:
await cwfs.latiss.take_darks(exptime=200, ndarks=3)

Generating group_id
DARK 0001 - 0003
Waiting for header service LFO before continuing
DARK 0002 - 0003
Waiting for header service LFO before continuing
DARK 0003 - 0003
Waiting for header service LFO before continuing


array([2020021800001, 2020021800002, 2020021800003])

In [20]:
await cwfs.latiss.take_bias(nbias=10)

Generating group_id
BIAS 0001 - 0010
Waiting for header service LFO before continuing
BIAS 0002 - 0010
Waiting for header service LFO before continuing
BIAS 0003 - 0010
Waiting for header service LFO before continuing
BIAS 0004 - 0010
Waiting for header service LFO before continuing
BIAS 0005 - 0010
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 23 messages
BIAS 0006 - 0010
Waiting for header service LFO before continuing
BIAS 0007 - 0010
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 30 messages
BIAS 0008 - 0010
Waiting for header service LFO before continuing
BIAS 0009 - 0010
Waiting for header service LFO before continuing
BIAS 0010 - 0010
Waiting for header service LFO before continuing


array([2020021800004, 2020021800005, 2020021800006, 2020021800007,
       2020021800008, 2020021800009, 2020021800010, 2020021800011,
       2020021800012, 2020021800013])

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 31 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages


In [ ]:
from lsst.rapid.analysis.bestEffort import BestEffortIsr
REPODIR = '/mnt/dmcs/oods_butler_repo/repo'
bestEffort = BestEffortIsr(REPODIR)
dataId = {'dayObs': '2020-02-17', 'seqNum': 244}
exp = bestEffort.getExposure(dataId)

In [1]:
print('test')

test
